# DeepDream Revived - Colab

Run the DeepDream algorithm on your images or videos directly in Google Colab.

### Features
- **Images & Videos**: Support for PNG, JPG, GIF, and MP4.
- **Octaves Mode**: Multi-scale processing for high-detail dreams.
- **Customizable**: Adjust steps, blend modes, and scales.


## 1. Install Dependencies
Install necessary libraries and tools.

In [ ]:
!apt-get install ffmpeg
!pip install tensorflow numpy pillow

import os
print("Dependencies installed.")

## 2. Deploy Code
Writing project files to Colab environment.

In [ ]:
%%writefile dream.py
import argparse
import random
import mimetypes
import os
import subprocess
import traceback
import sys

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2" # https://stackoverflow.com/questions/38073432/how-to-suppress-verbose-tensorflow-logging
os.environ["AUTOGRAPH_VERBOSITY"] = "0" # https://www.tensorflow.org/api_docs/python/tf/autograph/set_verbosity
import tensorflow
import numpy as np
import PIL.Image
import PIL.ImageChops
from tensorflow.keras import layers, models

ffmpeg_path = None # Edit this if you want to specify a custom path to ffmpeg
ffprobe_path = ffmpeg_path # Usually you should not have to edit this
ffmpeg_exe = os.path.join(ffmpeg_path, 'ffmpeg') if ffmpeg_path else 'ffmpeg'
ffprobe_exe = os.path.join(ffprobe_path, 'ffprobe') if ffprobe_path else 'ffprobe'

# Download an image and read it into a NumPy array.
def download(url, max_dim=None):
    name = url.split('/')[-1]
    image_path = tensorflow.keras.utils.get_file(name, origin=url)
    img = PIL.Image.open(image_path)
    if max_dim:
        img.thumbnail((max_dim, max_dim))
    return np.array(img)

def load_img(image_path : str, max_dim=None):
    img = PIL.Image.open(image_path)
    if max_dim:
        img.thumbnail((max_dim, max_dim))
    return np.array(img), img.size

def blend_img(next_image : str, prev_image : str, prev_dream_image : str, blend_amount=0.0, diff=False, max_dim=None):
    img = PIL.Image.open(next_image).convert('RGB')
    dream = PIL.Image.open(prev_dream_image).convert('RGB')
    if diff:
        prev = PIL.Image.open(prev_image).convert('RGB')
        if prev.size != dream.size:
            prev = prev.resize(dream.size)
        diff = PIL.ImageChops.difference(dream,prev)
        dream = diff
    # Images need to be the same size (this can happen when the dream has been scaled up or down
    if img.size != dream.size:
        dream = dream.resize(img.size)

    # Note: If blend is 0.0, a copy of image 1 is returned. If blend is 1.0, a copy of image 2 is returned.
    # Therefore the previous dream image in the sequence should be passed in as image 2
    blended = PIL.Image.blend(img, dream, blend_amount)
    if max_dim:
        blended.thumbnail((max_dim, max_dim))
    return np.array(blended), blended.size

# Normalize an image
def deprocess(img):
    img = 255*(img + 1.0)/2.0
    return tensorflow.cast(img, tensorflow.uint8)

def calc_loss(img, model):
    # Pass forward the image through the model to retrieve the activations.
    # Converts the image into a batch of size 1.
    img_batch = tensorflow.expand_dims(img, axis=0)
    layer_activations = model(img_batch)
    if len(layer_activations) == 1:
        layer_activations = [layer_activations]

    losses = []
    for act in layer_activations:
        loss = tensorflow.math.reduce_mean(act)
        losses.append(loss)

    return  tensorflow.reduce_sum(losses)

class DeepDream(tensorflow.Module):
    def __init__(self, model):
        self.model = model

    @tensorflow.function(
        input_signature=(
            tensorflow.TensorSpec(shape=[None,None,3], dtype=tensorflow.float32),
            tensorflow.TensorSpec(shape=[], dtype=tensorflow.int32),
            tensorflow.TensorSpec(shape=[], dtype=tensorflow.float32),)
    )
    def __call__(self, img, steps, step_size):
        loss = tensorflow.constant(0.0)
        for n in tensorflow.range(steps):
            with tensorflow.GradientTape() as tape:
                # This needs gradients relative to `img`
                # `GradientTape` only watches `tensorflow.Variable`s by default
                tape.watch(img)
                loss = calc_loss(img, self.model)

            # Calculate the gradient of the loss with respect to the pixels of the input image.
            gradients = tape.gradient(loss, img)

            # Normalize the gradients.
            gradients /= tensorflow.math.reduce_std(gradients) + 1e-8 

            # In gradient ascent, the "loss" is maximized so that the input image increasingly "excites" the layers.
            # You can update the image by directly adding the gradients (because they're the same shape!)
            img = img + gradients*step_size
            img = tensorflow.clip_by_value(img, -1, 1)

        return loss, img

def random_roll(img, maxroll):
    # Randomly shift the image to avoid tiled boundaries.
    shift = tensorflow.random.uniform(shape=[2], minval=-maxroll, maxval=maxroll, dtype=tensorflow.int32)
    img_rolled = tensorflow.roll(img, shift=shift, axis=[0,1])
    return shift, img_rolled

class TiledGradients(tensorflow.Module):
    def __init__(self, model):
        self.model = model

    @tensorflow.function(
        input_signature=(
            tensorflow.TensorSpec(shape=[None,None,3], dtype=tensorflow.float32),
            tensorflow.TensorSpec(shape=[2], dtype=tensorflow.int32),
            tensorflow.TensorSpec(shape=[], dtype=tensorflow.int32),)
    )
    def __call__(self, img, img_size, tile_size=512):
        shift, img_rolled = random_roll(img, tile_size)

        # Initialize the image gradients to zero.
        gradients = tensorflow.zeros_like(img_rolled)

        # Skip the last tile, unless there's only one tile.
        xs = tensorflow.range(0, img_size[1], tile_size)[:-1]
        if not tensorflow.cast(len(xs), bool):
            xs = tensorflow.constant([0])
        ys = tensorflow.range(0, img_size[0], tile_size)[:-1]
        if not tensorflow.cast(len(ys), bool):
            ys = tensorflow.constant([0])

        for x in xs:
            for y in ys:
                # Calculate the gradients for this tile.
                with tensorflow.GradientTape() as tape:
                    # This needs gradients relative to `img_rolled`.
                    # `GradientTape` only watches `tensorflow.Variable`s by default.
                    tape.watch(img_rolled)

                    # Extract a tile out of the image.
                    img_tile = img_rolled[y:y+tile_size, x:x+tile_size]
                    loss = calc_loss(img_tile, self.model)

                # Update the image gradients for this tile.
                gradients = gradients + tape.gradient(loss, img_rolled)

        # Undo the random shift applied to the image and its gradients.
        gradients = tensorflow.roll(gradients, shift=-shift, axis=[0,1])

        # Normalize the gradients.
        gradients /= tensorflow.math.reduce_std(gradients) + 1e-8 
        return gradients


def run_deep_dream_simple(img, dream_model, steps=100, step_size=0.01):
    # Convert from uint8 to the range expected by the model.
    img = tensorflow.keras.applications.inception_v3.preprocess_input(img)
    img = tensorflow.convert_to_tensor(img)
    step_size = tensorflow.convert_to_tensor(step_size)
    steps_remaining = steps
    step = 0
    
    # Check if we have cached the compiled function on the model
    if not hasattr(dream_model, 'deep_dream_func'):
        dream_model.deep_dream_func = DeepDream(dream_model)

    while steps_remaining:
        if steps_remaining>100:
            run_steps = tensorflow.constant(100)
        else:
            run_steps = tensorflow.constant(steps_remaining)
        steps_remaining -= run_steps
        step += run_steps

        loss, img = dream_model.deep_dream_func(img, run_steps, tensorflow.constant(step_size))
        
        display_img = deprocess(img)
        # print('Step {}, loss {}'.format(step, loss))

    result = deprocess(img)
    return result

def run_deep_dream_with_octaves(img, dream_model, steps_per_octave=100, step_size=0.01, 
                                octaves=None, octave_scale=1.3, tile_size=512):
    if octaves is None:
        octaves = range(-2,3)
    
    # If tile_size is 0 or None, set to max(512, max_dim) to avoid tiling if possible, or just huge number
    # If 0, user means auto/max.
    if not tile_size or tile_size <= 0:
        tile_size = max(img.shape[:2]) # Use full image size as one tile
        
    base_shape = tensorflow.cast(tensorflow.shape(img)[:-1], tensorflow.float32)
    img = tensorflow.keras.applications.inception_v3.preprocess_input(img)

    initial_shape = img.shape[:-1]
    img = tensorflow.image.resize(img, initial_shape)
    
    # Prepare compiled functions
    if not hasattr(dream_model, 'deep_dream_func'):
        dream_model.deep_dream_func = DeepDream(dream_model)
    if not hasattr(dream_model, 'tiled_gradients_func'):
        dream_model.tiled_gradients_func = TiledGradients(dream_model)

    for octave in octaves:
        # Scale the image based on the octave
        new_size = tensorflow.cast(base_shape * (octave_scale**octave), tensorflow.int32)
        img = tensorflow.image.resize(img, new_size)

        if tile_size is not None and (img.shape[0] > tile_size or img.shape[1] > tile_size):
             # Use TiledGradients if image is larger than tile_size
             # TiledGradients handles arbitrary size via tiling
             for step in range(steps_per_octave):
                 gradients = dream_model.tiled_gradients_func(img, new_size, tile_size=tile_size)
                 img = img + gradients*step_size
                 img = tensorflow.clip_by_value(img, -1, 1)
        else:
             # Use simple DeepDream if image fits in tile or no tile_size
             loss, img = dream_model.deep_dream_func(img, steps_per_octave, step_size)

    img = tensorflow.image.resize(img, initial_shape)
    result = deprocess(img)
    return result

def save_img(img, img_name : str):
    PIL.Image.fromarray(np.array(img)).save(img_name, 'PNG')

def get_input_framerate(input_filename : str):
    result = subprocess.run([ffprobe_exe,"-v", "error", "-select_streams", "v", "-of", "default=noprint_wrappers=1:nokey=1", "-show_entries", "stream=r_frame_rate", input_filename], stdout=subprocess.PIPE, text=True)
    if result.returncode != 0:
        print(result.stdout)
        print('ffprobe returned error code {}'.format(result.returncode))
        print('Error getting input fps.')
        return None
    # Outputs the frame rate as a precise fraction. Have to convert to decimal.
    source_fps_fractional = result.stdout.split('/')
    source_fps = round(float(source_fps_fractional[0]) / float(source_fps_fractional[1]), 2)
    return source_fps

def output_to_png_sequence(input_filename : str, output_dir : str):
    ffmpeg_args = ["ffmpeg", '-hide_banner', '-y', '-i', input_filename]
    framerate = get_input_framerate(input_filename)
    if framerate is not None:
        ffmpeg_args.extend(['-vf', f'fps={framerate}'])
    ffmpeg_args.append(os.path.join(output_dir,'%03d.png'))
    # Use popen so we can pend on completion
    result = subprocess.run(ffmpeg_args, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True, encoding='utf-8', errors='ignore')
    if result.returncode != 0:
        print(' '.join(ffmpeg_args))
        print(result.stderr)
        raise RuntimeError('ffmpeg returned error code {}'.format(result.returncode))
    png_files = [f for f in os.listdir(output_dir) if f.endswith('.png')]
    return sorted(png_files)

def concat_png_sequence(input_filename : str, png_dir : str, output_dir : str):
    mime, subtype = mimetypes.guess_type(input_filename)[0].split('/')
    framerate = get_input_framerate(input_filename)
    file_basename = os.path.splitext(os.path.basename(input_filename))[0]
    output_filename = os.path.join(output_dir,file_basename + ('.gif' if subtype == 'gif' else '.mp4'))
    ffmpeg_args = ["ffmpeg", '-hide_banner', '-y', '-framerate', f'{framerate}', '-pattern_type', 'glob', '-i', os.path.join(png_dir,'*.png')]
    if subtype != 'gif':
        ffmpeg_args.extend(['-c:v', 'libx264', '-crf', '20'])
    ffmpeg_args.append(output_filename)
    result = subprocess.run(ffmpeg_args, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True, encoding='utf-8', errors='ignore')
    if result.returncode != 0:
        print(' '.join(ffmpeg_args))
        print(result.stderr)
        raise RuntimeError('ffmpeg returned error code {}'.format(result.returncode))
    return output_filename

def load_inception_model(cpu_mode=False):
    if cpu_mode:
        tensorflow.config.set_visible_devices([], 'GPU')
        print('Using CPU only mode.')
    else:
        gpus = tensorflow.config.list_physical_devices('GPU')
        print('Num GPUs Available: ', len(gpus))
        if gpus:
            try:
                for gpu in gpus:
                    tensorflow.config.experimental.set_memory_growth(gpu, True)
            except RuntimeError as e:
                print(e)

    base_model = tensorflow.keras.applications.InceptionV3(include_top=False, weights='imagenet')
    names = ['mixed3', 'mixed5']
    layers = [base_model.get_layer(name).output for name in names]
    dream_model = tensorflow.keras.Model(inputs=base_model.input, outputs=layers)
    return dream_model

def process_file_or_directory(args, dream_model):
    print(f'Using DeepDream mode "{args.mode}", steps: {args.steps}, step size: {args.step_size}')
    if 'random' in args.octaves:
        num_octaves = int(args.octaves.split(' ')[-1])
        octaves = [random.randint(args.rand_min, args.rand_max) for _ in range(num_octaves)]
    elif 'range' in args.octaves:
        _, lower, upper = args.octaves.split(' ')
        octaves = range(int(lower), int(upper))
    else:
        octaves = [int(num) for num in args.octaves.split(',')]
    if args.mode == 'octaves':
        print(f'Octaves: {[octave for octave in octaves]}')
    
    os.makedirs(args.output, exist_ok=True)

    # DIRECTORY PROCESSING
    if os.path.isdir(args.input):
        print(f'Processing directory {args.input} (Worker {args.worker_id}/{args.num_workers})')
        files = sorted([f for f in os.listdir(args.input) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
        
        # Partition files
        my_files = files[args.worker_id::args.num_workers]
        print(f'Assigned {len(my_files)} files out of {len(files)} total.')

        output_dirname = os.path.join(args.output, os.path.basename(args.input.rstrip(os.sep)) + '-dream')
        os.makedirs(output_dirname, exist_ok=True)

        for filename in my_files:
            infile = os.path.join(args.input, filename)
            outfile = os.path.join(output_dirname, filename)
            print(f'  {infile} -> {outfile}')
            
            original_img, img_size = load_img(infile, max_dim=args.max_size)
            
            if args.mode == 'simple':
                dream_img = run_deep_dream_simple(original_img, dream_model, steps=args.steps, step_size=args.step_size)
            elif args.mode == 'octaves':
                dream_img = run_deep_dream_with_octaves(original_img, dream_model,
                    steps_per_octave=args.steps,
                    step_size=args.step_size,
                    octave_scale=args.scale,
                    tile_size=args.tile_size if args.tile_size is not None else max(img_size))
            else:
                raise RuntimeError(f'Unrecognized mode {args.mode}')
            
            save_img(dream_img, outfile)
        print(f'Worker {args.worker_id} finished.')
        if args.num_workers > 1: # Only exit if triggered as a worker process
            sys.exit(0)

    # SINGLE FILE PROCESSING
    else:
        guess = mimetypes.guess_type(args.input)[0]
        if guess:
            mime, subtype = guess.split('/')
        else:
            mime, subtype = None, None

        if mime == 'image' and subtype != 'gif':
            original_img, img_size = load_img(args.input, max_dim=args.max_size)
            if args.mode == 'simple':
                dream_img = run_deep_dream_simple(original_img, dream_model, steps=args.steps, step_size=args.step_size)
            elif args.mode == 'octaves':
                dream_img = run_deep_dream_with_octaves(original_img, dream_model,
                    steps_per_octave=args.steps,
                    step_size=args.step_size,
                    octave_scale=args.scale,
                    tile_size=args.tile_size if args.tile_size is not None else max(img_size))
            else:
                raise RuntimeError(f'Unrecognized mode {args.mode}')
            output_basename = os.path.splitext(os.path.basename(args.input))[0]
            output_filename = os.path.join(args.output,output_basename + '.png')
            save_img(dream_img, output_filename)
            print(f'Output rendered to {output_filename}')
        
        elif mime == 'video' or (mime == 'image' and subtype == 'gif'):
            file_basename = os.path.splitext(os.path.basename(args.input))[0]
            output_dirname = os.path.join(args.output, file_basename)
            png_dir = os.path.join(output_dirname, 'pngs')
            png_dream_dir = os.path.join(output_dirname, 'pngs-dream')
            os.makedirs(png_dir, exist_ok=True)
            os.makedirs(png_dream_dir, exist_ok=True)

            print(f'Extracting frames to {png_dir}')
            output_to_png_sequence(args.input, png_dir)
            
            files = sorted([f for f in os.listdir(png_dir) if f.endswith('.png')])
            print(f'Processing {len(files)} frames...')

            previous_img = None
            for i, filename in enumerate(files):
                infile = os.path.join(png_dir, filename)
                outfile = os.path.join(png_dream_dir, filename)
                
                original_img, img_size = load_img(infile, max_dim=args.max_size)
                
                # Blend with previous frame if enabled
                if args.blend > 0 and previous_img is not None:
                     # Resize previous to match current (if video size changes, though unlikely for mp4)
                     # Simple blend: input = current * (1-blend) + prev * blend
                     # But deepdream logic usually blends *before* processing to create flow.
                     # Here let's just use the logic as intended: modify 'original_img'
                     # Note: Tensors in TF are immutable, need to act carefully.
                     # load_img returns a tensor. 
                     # We can just do weighted average.
                     original_img = original_img * (1.0 - args.blend) + previous_img * args.blend
                
                if args.mode == 'simple':
                    dream_img = run_deep_dream_simple(original_img, dream_model, steps=args.steps, step_size=args.step_size)
                elif args.mode == 'octaves':
                    dream_img = run_deep_dream_with_octaves(original_img, dream_model,
                        steps_per_octave=args.steps,
                        step_size=args.step_size,
                        octave_scale=args.scale,
                        tile_size=args.tile_size if args.tile_size is not None else max(img_size))
                
                save_img(dream_img, outfile)
                print(f'{infile} -> {outfile}', flush=True)
                
                # Update previous_img for next frame
                # Optional: use dream_img or original_img? 
                # Flow works best if we feed the *result* back, or at least the hallucinated structure.
                # Standard implementation feeds the *result*.
                # But we need to ensure size matches.
                previous_img = dream_img 

            print('Creating output video...')
            output_video = os.path.join(args.output, file_basename + '-dream.mp4')
            concat_png_sequence(args.input, png_dream_dir, output_video)
            print(f'Video saved to {output_video}')
        else:
            print(f'Unsupported file type: {mime}/{subtype}')

if __name__ == "__main__":
    try:
        parser = argparse.ArgumentParser(prog='DeepDream runner')
        parser.add_argument('-i', '--input', type=str, default='example.png', help='Input file path (Image or Video) or Directory')
        parser.add_argument('--output', type=str, default='output', help='Output directory path')
        parser.add_argument('--mode', type=str, choices=['simple', 'octaves'], default='simple', help='DeepDream mode')
        parser.add_argument('--octaves', type=str, default='-2,-1,0,1,2', help='Octaves/Layers to use. Examples: "random 5" (5 random layers), "range 3 8" (layers 3 to 7), "-2,-1,0,1,2" (specific layers)')
        parser.add_argument('--rand_min', type=int, default=1, help='Min layer index for random octaves')
        parser.add_argument('--rand_max', type=int, default=11, help='Max layer index for random octaves')
        parser.add_argument('--scale', type=float, default=1.3, help='Scale factor for octaves')
        parser.add_argument('--max_size', type=int, default=0, help='Max image dimension (0 = no limit)')
        parser.add_argument('--blend', type=float, default=0.0, help='Amount to blend images when processing video')
        parser.add_argument('--cpu', action='store_true', help='Use CPU only')
        parser.add_argument('--diff', action='store_true', help='Use difference between original and dream image')
        parser.add_argument('--steps', type=int, default=20, help='Total number of steps, or steps per octave if using "octaves" mode')
        parser.add_argument('--step_size', type=float, default=0.1, help='Step size')
        parser.add_argument('--tile_size', type=int, help='Tile size used for "octaves" mode. If not specified, the tile size is the max dimension of the input image.')
        parser.add_argument('--worker_id', type=int, default=0, help='Worker ID for parallel processing (0-based)')
        parser.add_argument('--num_workers', type=int, default=1, help='Total number of workers for parallel processing')

        args, unknown_args = parser.parse_known_args()
        
        model = load_inception_model(args.cpu)
        process_file_or_directory(args, model)
    except Exception:
        print(traceback.format_exc())

## 3. Setup Input Source
Choose to upload a file or mount Google Drive.

In [ ]:
import os
from google.colab import files
from google.colab import drive

source_type = "Upload File" #@param ["Mount Google Drive", "Upload File"]
mount_path = "/content/drive" 

if source_type == "Mount Google Drive":
    drive.mount(mount_path)
    print(f"Google Drive mounted at {mount_path}")
elif source_type == "Upload File":
    uploaded = files.upload()
    if uploaded:
        # Taking the first uploaded file
        fname = list(uploaded.keys())[0]
        print(f"File uploaded to: /content/{fname}")
        print("Please copy this path to the 'input_path' field below if it's not automatically set.")


## 4.0 Preview Frame (Optional)
Helper to extract a single frame from a video to check if the input path is correct or to decide on parameters.

In [ ]:
input_path = "example.png" #@param {type:"string"}
preview_frame_index = 0 #@param {type:"integer"}

import os
import subprocess
from IPython.display import Image, display

if not os.path.exists(input_path):
    print(f"Error: Input file at {input_path} not found.")
else:
    # Check mime type simply by extension or assuming it's supported
    import mimetypes
    mime, _ = mimetypes.guess_type(input_path)
    
    if mime and mime.startswith('image') and not input_path.endswith('.gif'):
        print("Input is an image, displaying directly:")
        display(Image(input_path))
    else:
        # Extract frame using ffmpeg
        output_preview = "preview.jpg"
        # -vf select='eq(n\,{index})' -vframes 1
        cmd = [
            "ffmpeg", "-y", "-hide_banner", "-loglevel", "error",
            "-i", input_path,
            "-vf", f"select='eq(n,{preview_frame_index})'",
            "-vframes", "1",
            output_preview
        ]
        print(f"Extracting frame {preview_frame_index}...")
        subprocess.run(cmd)
        if os.path.exists(output_preview):
             display(Image(output_preview))
        else:
             print("Failed to extract frame. Is the index out of bounds?")


## 4.0 Load Model
Load the InceptionV3 model once to speed up subsequent runs.

In [ ]:
try:
    import dream
    if 'dream_model' not in globals() or dream_model is None:
        print("Loading InceptionV3 Model...")
        dream_model = dream.load_inception_model()
        print("Model loaded successfully.")
    else:
        print("Model already loaded.")
except ImportError:
    print("Error: dream.py not found. Please ensure 'Deploy Code' step ran successfully.")


## 4.1 Run DeepDream


In [ ]:

input_path = "example.png" #@param {type:"string"}
output_dir = "output" #@param {type:"string"}

#@markdown ### Processing Mode
mode = "octaves" #@param ["simple", "octaves"]
octaves = "-2, -1, 0, 1, 2" #@param {type:"string"}
scale = 1.3 #@param {type:"number"}
tile_size = 0 #@param {type:"integer"}

#@markdown ### Iteration Settings
steps = 20 #@param {type:"integer"}
step_size = 0.1 #@param {type:"number"}

#@markdown ### Advanced & Video
blend = 0.0 #@param {type:"number"}
diff = False #@param {type:"boolean"}
max_size = 0 #@param {type:"integer"}
use_cpu = False #@param {type:"boolean"}
max_workers = 1 #@param {type:"slider", min:1, max:4, step:1}

#@markdown ### Preview (Video only)
preview_video_frame = False #@param {type:"boolean"}
preview_frame_num = 1 #@param {type:"integer"}

import subprocess
import os
import sys
import shutil
import re
from IPython.display import Image, display
from PIL import Image as PILImage
import dream # Ensure available
import argparse # Needed for constructing args object

def get_video_info(filename):
    if not os.path.exists(filename):
        return None
    cmd = ["ffprobe", "-v", "error", "-select_streams", "v", "-show_entries", "stream=width,height,display_aspect_ratio,r_frame_rate", "-of", "csv=p=0", filename]
    try:
        result = subprocess.run(cmd, stdout=subprocess.PIPE, text=True)
        if result.returncode == 0:
            parts = result.stdout.strip().split(',')
            if len(parts) >= 2:
                w, h = parts[0], parts[1]
                ar = parts[2] if len(parts) > 2 else f"{w}:{h}"
                fps_str = parts[3] if len(parts) > 3 else "30/1"
                try:
                    num, den = fps_str.split('/')
                    fps = float(num) / float(den)
                except:
                    fps = 30.0
                return w, h, ar, fps
    except Exception:
        pass
    return None

if not os.path.exists(input_path):
    print(f"Error: Input file not found at {input_path}")
else:
    # Check for model
    if 'dream_model' not in globals():
        print("Model not loaded. Attempting to load (this might take a moment)...")
        dream_model = dream.load_inception_model(use_cpu)

    # Construct Args object to pass to dream.process_file_or_directory
    # We simulate argparse namespace
    class DreamArgs:
        def __init__(self):
             self.input = input_path
             self.output = output_dir
             self.mode = mode
             self.octaves = octaves
             self.scale = scale
             self.steps = steps
             self.step_size = step_size
             self.tile_size = tile_size if tile_size > 0 else None
             self.blend = blend
             self.diff = diff
             self.max_size = max_size
             self.cpu = use_cpu
             self.rand_min = 1
             self.rand_max = 11
             self.worker_id = 0
             self.num_workers = 1

    run_args = DreamArgs()

    # Handle Preview Mode logic
    if preview_video_frame:
        if max_workers > 1:
            print("Warning: Preview disabled in Multi-Worker mode.")
        else:
            print("--- PREVIEW MODE ---")
            v_info = get_video_info(input_path)
            if v_info:
                 print(f"Original Video: {v_info[0]}x{v_info[1]} (AR: {v_info[2]}, FPS: {v_info[3]:.2f})")
            
            preview_img = "preview_temp.png"
            print(f"Extracting frame {preview_frame_num}...")
            frame_idx = max(0, preview_frame_num - 1)
            # FIXED: Syntax Warning escape sequence
            cmd_extract = [
                "ffmpeg", "-y", "-i", input_path,
                "-vf", f"select='eq(n,{frame_idx})'",
                "-vframes", "1", preview_img
            ]
            subprocess.run(cmd_extract, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
            
            if os.path.exists(preview_img):
                print("Processing preview frame...")
                # Update args for preview
                preview_args = DreamArgs()
                preview_args.input = preview_img
                # Use loaded model directly!
                dream.process_file_or_directory(preview_args, dream_model)
                
                result_path = os.path.join(output_dir, "preview_temp.png")
                if os.path.exists(result_path):
                    with PILImage.open(result_path) as img:
                        w, h = img.size
                        ar = f"{w/h:.2f}"
                        print(f"Processed Frame: {w}x{h} (AR: ~{ar})")
                    display(Image(result_path))
                else:
                    print("Error: Preview processing failed.")
            else:
                print("Error: Could not extract frame.")
            # No sys.exit(0) here because we are in main process, just stop doing other things

    # Multi-Worker Setup (Still uses subprocess for isolation/GIL)
    elif max_workers > 1:
        # Check if input is video
        import mimetypes
        fake_mime, _ = mimetypes.guess_type(input_path)
        if fake_mime and fake_mime.startswith('image') and not input_path.endswith('.gif'):
            print("Multi-worker mode ignored for single image input.")
            max_workers = 1
            # Fallthrough to Normal Processing
            is_multi_worker = False

        else:
             is_multi_worker = True
             if blend > 0:
                 print("Warning: Blend > 0 is incompatible with Multi-Worker mode. Enforcing blend=0.")
                 blend = 0.0

             print(f"--- MULTI-WORKER MODE ({max_workers} Parallel Processes) ---")
             
             # 1. Extract Frames
             v_info = get_video_info(input_path)
             fps = v_info[3] if v_info else 30.0
             print(f"Input FPS: {fps:.2f}")
             
             # Create structured output folders: output/video_name/pngs
             base_name = os.path.splitext(os.path.basename(input_path))[0]
             video_output_dir = os.path.join(output_dir, base_name)
             temp_frames_dir = os.path.join(video_output_dir, "pngs") 
             processed_dir = os.path.join(video_output_dir, "pngs-dream")
             
             if os.path.exists(temp_frames_dir): shutil.rmtree(temp_frames_dir)
             if os.path.exists(processed_dir): shutil.rmtree(processed_dir)
             os.makedirs(temp_frames_dir, exist_ok=True)
             os.makedirs(processed_dir, exist_ok=True)
             
             print("Extracting frames for parallel processing...")
             cmd_extract = ["ffmpeg", "-hide_banner", "-loglevel", "error", "-i", input_path, os.path.join(temp_frames_dir, "%05d.png")]
             subprocess.run(cmd_extract, check=True)
             
             # 2. Launch Workers
             print(f"Launching {max_workers} 'dream.py' workers...")
             procs = []
             for i in range(max_workers):
                 # Point workers to write to the correct processed_dir directly?
                 # dream.py logic says: output_dirname = os.path.join(args.output, base + '-dream') if dir input
                 # If input is .../pngs, output will be .../pngs-dream. Perfect.
                 # So we pass video_output_dir as 'output', and input as 'temp_frames_dir'.
                 # dream.py will create video_output_dir/pngs-dream
                 
                 cmd = ["python", "-u", "dream.py", "-i", temp_frames_dir, "--output", video_output_dir]
                 # Add common params
                 cmd.extend(["--mode", mode, "--worker_id", str(i), "--num_workers", str(max_workers)])
                 if mode == "octaves":
                     cmd.extend(["--octaves", octaves, "--scale", str(scale)])
                     if tile_size > 0: cmd.extend(["--tile_size", str(tile_size)])
                 cmd.extend(["--steps", str(steps), "--step_size", str(step_size)])
                 if max_size > 0: cmd.extend(["--max_size", str(max_size)])
                 if use_cpu: cmd.append("--cpu")
                 
                 # Use PIPE to avoid fileno error in Colab
                 p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1, universal_newlines=True)
                 procs.append(p)
             
             # Monitor workers and print output
             import time
             try:
                 while any(p.poll() is None for p in procs):
                     for i, p in enumerate(procs):
                         # Read available lines
                         while True:
                             line = p.stdout.readline()
                             if not line: break
                             print(f"[Worker {i}] {line.strip()}")
                     time.sleep(0.1)
             except KeyboardInterrupt:
                 print("\nInterrupted! Killing workers...")
                 for p in procs:
                     p.terminate()
                 # Wait a bit for them to die
                 time.sleep(1)
                 for p in procs:
                     if p.poll() is None:
                         p.kill()
                 print("Workers terminated.")
                 raise # Re-raise to stop cell execution
             
             # Final flush
             for i, p in enumerate(procs):
                 for line in p.stdout:
                     print(f"[Worker {i}] {line.strip()}")
                 if p.returncode != 0:
                     print(f"Worker {i} failed with code {p.returncode}")
             
             print("All workers finished.")
             
             # 3. Combine Frames
             # dream.py creates folder "input_dirname-dream" inside output dir.
             # input was "video_output_dir/pngs", so output is "video_output_dir/pngs-dream"
             # which matches 'processed_dir' variable we defined above.
             
             if not os.path.exists(processed_dir):
                 print(f"Error: Processed frames directory not found at {processed_dir}. Did workers fail?")
             else:
                 print("Combining processed frames...")
                 output_filename = os.path.join(output_dir, base_name + "-dream.mp4")
                 cmd_combine = [
                     "ffmpeg", "-y", "-framerate", str(fps), "-pattern_type", "glob",
                     "-i", os.path.join(processed_dir, "*.png"),
                     "-c:v", "libx264", "-pix_fmt", "yuv420p",
                     "-vf", "pad=ceil(iw/2)*2:ceil(ih/2)*2", # Fix div-by-2 error
                     output_filename
                 ]
                 subprocess.run(cmd_combine, check=True)
                 print(f"Processing Complete. Video saved to {output_filename}")

    # Fallback Logic Check: execute only if NOT multi-worker AND not preview
    # We initialize is_multi_worker = False at top of this cell implicitly by not setting it? 
    # Better to rely on locals() check carefully or just 'else' branch? 
    # But previous block was elif.
    # Logic:
    # if preview: ...
    # elif max_workers > 1: ... (sets is_multi_worker=True unless single image)
    # 
    # We need a final block that runs for single worker.
    # It should run if:
    # 1. Not preview
    # 2. AND (max_workers == 1 OR (max_workers > 1 but downgraded to single image))
    
    should_run_single = False
    if not preview_video_frame:
        if max_workers == 1:
            should_run_single = True
        elif 'is_multi_worker' in locals() and not is_multi_worker:
             should_run_single = True
        # If is_multi_worker is True, we do NOT run single.
        
    if should_run_single:
        # --- STANDARD SINGLE WORKER MODE (Video or Image) ---
        # Use imported dream module for speed!
        print(f"Processing {input_path} in single-process mode...")
        dream.process_file_or_directory(run_args, dream_model)
        
        print("\nProcessing Complete.")
        # Check if it was a single image and display it
        import mimetypes
        mime, subtype = mimetypes.guess_type(input_path)
        if mime and mime.startswith('image') and not subtype == 'gif':
             base = os.path.splitext(os.path.basename(input_path))[0]
             out_file = os.path.join(output_dir, base + '.png')
             if os.path.exists(out_file):
                 print("Result:")
                 display(Image(out_file))


## 4.2 Create Video from Frames (Optional)
Run this cell if you want to manually create a video from a folder of images. Useful if `dream.py` stopped early or if you want to adjust framerate.

In [ ]:
frames_folder = "output/example" #@param {type:"string"}
output_video_filename = "output/dream.mp4" #@param {type:"string"}
#@markdown Leave fps as 0 to attempt auto-detect from input_path (if it exists)
fps = 0 #@param {type:"integer"}
input_path_for_fps = "example.png" #@param {type:"string"}
transfer_audio = True #@param {type:"boolean"}
#@markdown ### Resolution
#@markdown -1: Keep processed resolution (safest). 0: Match original input resolution. >0: Scale width (keep aspect).
video_resolution = -1 #@param {type:"integer"}

#@markdown ### Save to Drive
save_to_drive = False #@param {type:"boolean"}
drive_output_path = "/content/drive/MyDrive" #@param {type:"string"}

import os
import subprocess
import shutil

def get_fps(filename):
    if not filename or not os.path.exists(filename):
        return None
    cmd = ["ffprobe", "-v", "error", "-select_streams", "v", "-of", "default=noprint_wrappers=1:nokey=1", "-show_entries", "stream=r_frame_rate", filename]
    try:
        result = subprocess.run(cmd, stdout=subprocess.PIPE, text=True)
        if result.returncode == 0:
            num, den = result.stdout.strip().split('/')
            return float(num) / float(den)
    except Exception:
        pass
    return None

def get_resolution(filename):
    if not filename or not os.path.exists(filename):
        return None
    cmd = ["ffprobe", "-v", "error", "-select_streams", "v", "-of", "csv=p=0", "-show_entries", "stream=width,height", filename]
    try:
        result = subprocess.run(cmd, stdout=subprocess.PIPE, text=True)
        if result.returncode == 0:
            w, h = map(int, result.stdout.strip().split(','))
            return w, h
    except Exception:
        pass
    return None

def has_audio_stream(filename):
    if not filename or not os.path.exists(filename):
        return False
    cmd = ["ffprobe", "-v", "error", "-select_streams", "a", "-show_entries", "stream=index", "-of", "csv=p=0", filename]
    try:
        result = subprocess.run(cmd, stdout=subprocess.PIPE, text=True)
        if result.returncode == 0 and len(result.stdout.strip()) > 0:
            return True
    except Exception:
        pass
    return False

if not os.path.exists(frames_folder):
    print(f"Error: Folder {frames_folder} not found.")
else:
    # Check if folder has pngs
    pngs = [f for f in os.listdir(frames_folder) if f.endswith('.png')]
    if not pngs:
        print(f"No .png files found in {frames_folder}")
    else:
        print(f"Found {len(pngs)} frames.")
        
        target_fps = fps
        if target_fps <= 0:
            # Try to detect
            detected = get_fps(input_path_for_fps)
            if detected:
                print(f"Auto-detected FPS from {input_path_for_fps}: {detected}")
                target_fps = detected
            else:
                print("Could not auto-detect FPS, defaulting to 30.")
                target_fps = 30
        else:
             print(f"Using manual FPS: {target_fps}")

        print("Combining frames...")
        
        # Build ffmpeg command
        cmd = ["ffmpeg", "-y"]
        
        # Input 0: Frames with sequence pattern
        cmd.extend([
            "-f", "image2",
            "-framerate", str(target_fps),
            "-start_number", "1",
            "-i", os.path.join(frames_folder, "%05d.png")
        ])
        
        # Input 1: Audio (original video)
        audio_source = None
        if transfer_audio and has_audio_stream(input_path_for_fps):
            audio_source = input_path_for_fps
            print(f"Audio detected in {audio_source}, will copy to output.")
            # Input 2: Audio Source
            cmd.extend(["-i", audio_source])
        
        # Output Options
        cmd.extend(["-c:v", "libx264", "-pix_fmt", "yuv420p"])
        
        # Filters (Resolution & Pad)
        filters = []
        
        if video_resolution == 0:
             # Match original
             orig_dims = get_resolution(input_path_for_fps)
             if orig_dims:
                 w, h = orig_dims
                 print(f"Scaling to match original resolution: {w}x{h}")
                 filters.append(f"scale={w}:{h}")
             else:
                 print("Could not determine original resolution to match. Using default.")
        elif video_resolution > 0:
             print(f"Scaling width to {video_resolution} (keeping aspect)")
             filters.append(f"scale={video_resolution}:-2")
        
        # Always apply pad to ensure mult of 2
        filters.append("pad=ceil(iw/2)*2:ceil(ih/2)*2")
        
        cmd.extend(["-vf", ",".join(filters)])
        
        if audio_source:
            # Map video from stream 0, audio from stream 1
            cmd.extend(["-map", "0:v", "-map", "1:a", "-c:a", "copy", "-shortest"])
        
        cmd.append(output_video_filename)

        print(f"Executing: {' '.join(cmd)}")
        process = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
        if process.returncode == 0:
            print(f"Video created successfully: {output_video_filename}")
            if save_to_drive:
                if not os.path.exists(drive_output_path):
                    print(f"Drive path {drive_output_path} does not exist. Attempting to create...")
                    try:
                         os.makedirs(drive_output_path, exist_ok=True)
                    except Exception as e:
                         print(f"Could not create Drive directory: {e}")
                
                if os.path.exists(drive_output_path):
                    try:
                        dest_file = os.path.join(drive_output_path, os.path.basename(output_video_filename))
                        shutil.copy2(output_video_filename, dest_file)
                        print(f"Saved to Drive: {dest_file}")
                    except Exception as e:
                        print(f"Error saving to Drive: {e}")
                else:
                     print("Skipping Drive save due to invalid path.")

        else:
            print("Error creating video:")
            print(process.stderr)


## 5. View/Download Results


In [ ]:
import os
from google.colab import files
from IPython.display import Image, display

# Try to find the output image
input_basename = os.path.splitext(os.path.basename(input_path))[0]
possible_output = os.path.join(output_dir, input_basename + ".png")

if os.path.exists(possible_output):
    print(f"Displaying result: {possible_output}")
    display(Image(possible_output))
    
    # Checkbox to download
    download_result = True #@param {type:"boolean"}
    if download_result:
        files.download(possible_output)
else:
    print(f"Output file {possible_output} not found. (If you processed a video, check the output folder manually)")
